In [1]:
import os
import sys
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from scipy.io import mmwrite, mmread
import seaborn as sns
import pickle
import utils
from importlib import reload
reload(utils)
%matplotlib inline
sns.set(font='IPAexGothic')

In [2]:
# 一つ上の階層のmoduleをインポートできるようにする
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append( str(current_dir) + '/../' )

In [3]:
from setting_param import MakeSample_MakeGraph_InputDir, MakeSample_MakeGraph_OutputDir
InputDir = MakeSample_MakeGraph_InputDir
OutputDir = MakeSample_MakeGraph_OutputDir

In [4]:
os.mkdir(OutputDir)

In [5]:
_ = pickle.load(open(InputDir + "/data.pickle","rb"))
cluster_sizes = _["cluster_sizes"]
cluster2id = _["cluster2id"]
locations = _["locations"]
timestamps = _["timestamps"]
raw_data = _["raw_data"]
cluster_data = _["cluster_data"]
onehot_data = _["onehot_data"]

In [6]:
in_size = 48
out_size = 8

data = utils.data_iterator(cluster_data,timestamps,in_size,out_size=out_size,slide=1)
attr = data[0][1]

In [7]:
attr.shape

(89014, 8, 4)

In [8]:
cluster_sizes

array([10, 15,  5, 13,  6,  6, 14, 15,  6, 20, 12,  6, 10,  5, 18, 10, 10,
       11, 16, 29,  7, 12, 10, 10, 18, 17, 12])

In [9]:
cluster2id

{0: [10, 11, 12, 13, 14, 15, 310, 311, 312, 313],
 1: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
 2: [31, 32, 33, 34, 35],
 3: [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
 4: [101, 102, 103, 104, 105, 106],
 5: [107, 108, 109, 110, 111, 112],
 6: [58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71],
 7: [113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127],
 8: [299, 300, 301, 302, 303, 304],
 9: [203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187],
 10: [213, 214, 215, 216, 217, 218, 188, 189, 190, 191, 192, 193],
 11: [219, 220, 194, 195, 196, 197],
 12: [221, 222, 223, 224, 225, 198, 199, 200, 201, 202],
 13: [318, 319, 320, 321, 322],
 14: [290,
  291,
  292,
  293,
  294,
  295,
  296,
  297,
  298,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146],
 15: [152, 153, 154, 155, 156, 147, 148, 149, 150, 15

In [10]:
cluster_list = list(range(27))
cluster_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26]

In [11]:
locations

array([[43.46304594, -3.79902855],
       [43.46305129, -3.79897021],
       [43.46305567, -3.79892327],
       [43.46306005, -3.79887834],
       [43.46306638, -3.79881665],
       [43.46307271, -3.79875228],
       [43.46307903, -3.79868656],
       [43.4630839 , -3.79862018],
       [43.4630912 , -3.7985491 ],
       [43.46309607, -3.79849143],
       [43.4628936 , -3.80350515],
       [43.46289944, -3.80344816],
       [43.46290723, -3.80339049],
       [43.46291307, -3.80332947],
       [43.46291794, -3.80326845],
       [43.46292524, -3.80320542],
       [43.46300213, -3.80242355],
       [43.463007  , -3.80236924],
       [43.46301041, -3.80231157],
       [43.46301771, -3.80223848],
       [43.46302355, -3.80217746],
       [43.46302988, -3.80212247],
       [43.4630362 , -3.80205944],
       [43.46304253, -3.80201183],
       [43.46304886, -3.80195349],
       [43.4630547 , -3.80188644],
       [43.46306151, -3.80182341],
       [43.46306492, -3.80176306],
       [43.46307222,

In [12]:
raw_data.shape # そのままの値

(109514, 27)

In [13]:
cluster_data.shape # 正規化された値。 raw_data/cluster_sizes

(109514, 27)

In [14]:
# clusterごとのlocationsを求める
location_dic = {}
for cid, node_id_list in cluster2id.items():
    latitude = 0
    longitude = 0
    for node_id in node_id_list:
        latitude += locations[node_id][0]
        longitude += locations[node_id][1]
    location_dic[cid] = (latitude/cluster_sizes[cid], longitude/cluster_sizes[cid])
location_dic

{0: (43.462934984, -3.8031018662999996),
 1: (43.46304220533333, -3.8020056188666667),
 2: (43.463135002399994, -3.8010452985999996),
 3: (43.463221747, -3.8001626557692303),
 4: (43.46268375016666, -3.800753206),
 5: (43.46244632016666, -3.8008664175),
 6: (43.46293138835713, -3.800218201214286),
 7: (43.462512025333325, -3.800186678733333),
 8: (43.46428866583333, -3.800416771833333),
 9: (43.464058553950004, -3.7997886166499995),
 10: (43.46354507466666, -3.799695840666667),
 11: (43.4631804555, -3.7996172718333328),
 12: (43.4628070155, -3.7995602264999997),
 13: (43.46396602240001, -3.7994388162),
 14: (43.46417569383334, -3.7985531225555556),
 15: (43.4636775221, -3.7984617774),
 16: (43.463307183199994, -3.7983939859999998),
 17: (43.46294807636364, -3.798330687454545),
 18: (43.464407522625, -3.7990541931874997),
 19: (43.46457756648275, -3.7976750693793107),
 20: (43.46386252214286, -3.798936424),
 21: (43.46340172608333, -3.799055088333333),
 22: (43.463070222999995, -3.79877

In [15]:
node_idx_table = {}
for idx, node in enumerate(cluster_list):
    node_idx_table[node] = idx
node_idx_table

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26}

In [16]:
rows = []
for node, idx in node_idx_table.items():
    rows.append([node, idx])
df = pd.DataFrame(rows, columns=['node', 'ID'])
df.to_csv(OutputDir + '/node_idx.csv', index=False, encoding='utf_8_sig')

In [17]:
columns = ['name', 'ID', 'range0', 'range1', 'type', 'primary', 'mean', 'std', 'min', 'max', 'class0', 'class1']

rows = []
row = ['parking type', 0, 0, 1, 'VECTOR', True, False, False, False, False, 'dummy0', 'dummy1']
rows.append(row)
row = ['available num', 1, 2, 2, 'SCALAR', False, cluster_data.mean(), cluster_data.std(), cluster_data.min(), cluster_data.max(), False, False]
rows.append(row)
row = ['attr(timestamp)', 2, 3, 34, 'VECTOR', False, False, False, False, False, False, False]
rows.append(row)

df = pd.DataFrame(rows, columns=columns)
df.to_csv(OutputDir + '/attribute_idx.csv', index=False, encoding='utf_8_sig')
df

,name,ID,range0,range1,type,primary,mean,std,min,max,class0,class1
0,parking type,0,0,1,VECTOR,True,False,False,False,False,dummy0,dummy1
1,available num,1,2,2,SCALAR,False,0.83267,0.179818,0,1,False,False
2,attr(timestamp),2,3,34,VECTOR,False,False,False,False,False,False,False


In [18]:
#共通設定

train_start, train_end = 8500, -15000
valid_start, valid_end = -15000,-13000
test_start, test_end  = -13000,-12000

in_size = 48
out_size = 8
interval = 15

In [19]:
trs, tre = list(range(109514))[train_start], list(range(109514))[train_end]

In [20]:
vas, vae = list(range(109514))[valid_start], list(range(109514))[valid_end]

In [21]:
tes, tee = list(range(109514))[test_start], list(range(109514))[test_end]

In [22]:
tre - trs

86014

In [23]:
vae - vas

2000

In [24]:
tee - tes

1000

In [39]:
tee - trs

89014

In [25]:
dates = timestamps[trs:tee]

In [26]:
rows = []
for idx, date in enumerate(dates):
    rows.append([date, idx])
df = pd.DataFrame(rows, columns=['TS', 'ID'])
df.to_csv(OutputDir + '/ts_idx.csv', index=False, encoding='utf_8_sig')

In [27]:
# pandas 高速化 https://kunai-lab.hatenablog.jp/entry/2018/04/08/134924
df = pd.read_csv(OutputDir + '/ts_idx.csv')
df_TS = df.TS.values
df_ID = df.ID.values
ts_idx_table = {}
for row in range(df.shape[0]):
    ts_idx_table[df_TS[row]] = df_ID[row]

In [28]:
len(ts_idx_table)

89014

In [29]:
len(node_idx_table)

27

In [30]:
exist_table = np.ones((len(node_idx_table), len(ts_idx_table)))
np.save(OutputDir + '/exist_table', exist_table)

In [31]:
for ts in range(len(ts_idx_table)):
    node_attribute = np.zeros((len(node_idx_table), 35))
    for node_id in range(len(node_idx_table)):
        node_attribute[node_id][0] = 1 # primary class、dummy0を表す。
        node_attribute[node_id][2] = cluster_data[train_start + ts][node_id]
        attr_flatten = attr[ts].reshape(-1)
        node_attribute[node_id][3:35] = attr_flatten
    np.save(OutputDir + '/node_attribute' + str(ts), node_attribute)

In [32]:
# 緯度・経度から2点間の距離を求める
#https://qiita.com/s-wakaba/items/e12f2a575b6885579df7
from math import sin, cos, acos, radians
earth_rad = 6378.137

def latlng_to_xyz(lat, lng):
    rlat, rlng = radians(lat), radians(lng)
    coslat = cos(rlat)
    return coslat*cos(rlng), coslat*sin(rlng), sin(rlat)

def dist_on_sphere(pos0, pos1, radius=earth_rad):
    xyz0, xyz1 = latlng_to_xyz(*pos0), latlng_to_xyz(*pos1)
    return acos(sum(x * y for x, y in zip(xyz0, xyz1)))*radius

In [33]:
dist_list =[]
for cluster_1 in cluster_list:
    for cluster_2 in cluster_list[cluster_list.index(cluster_1):]:
        if cluster_1 == cluster_2:
            continue
        dist = dist_on_sphere(location_dic[cluster_1], location_dic[cluster_2])
        dist_list.append(dist)
dist_list = np.array(dist_list)
print("距離パーセンタイル")
print("000%", np.percentile(dist_list, 0))
print("025%", np.percentile(dist_list, 25))
print("050%", np.percentile(dist_list, 50))
print("075%", np.percentile(dist_list, 75))
print("100%", np.percentile(dist_list, 100))

距離パーセンタイル
000% 0.027968740033382622
025% 0.09421652522933369
050% 0.14449681508385825
075% 0.19965567407899545
100% 0.47506683288359797


In [34]:
station_list = cluster_list
coordinates = location_dic
flag25, flag50, flag75, flag100 = 0, 0, 0, 0
adj_max_sum = len(node_idx_table) * len(node_idx_table) -  len(node_idx_table)
print("隣接行列dense率と距離閾値")
for thres in range(1000):
    thres = thres / 1000.0
    adj = np.zeros((len(node_idx_table), len(node_idx_table)))
    for station_1 in station_list:
        for station_2 in station_list:
            if station_1 == station_2:
                continue
            dist = dist_on_sphere(coordinates[station_1], coordinates[station_2])
            if dist < thres:
                adj[node_idx_table[station_1]][node_idx_table[station_2]] = 1
    dense_rate = adj.sum() / adj_max_sum
    if flag25==0 and dense_rate > 0.25:
        flag25 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)
    if flag50==0 and dense_rate > 0.5:
        flag50 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)
    if flag75==0 and dense_rate > 0.75:
        flag75 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)
    if flag100==0 and dense_rate >= 1:
        flag100 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)

隣接行列dense率と距離閾値
25%,  < 0.095
50%,  < 0.145
75%,  < 0.2
100%,  < 0.476


In [35]:
#dist_matrix = np.full((len(node_idx_table), len(node_idx_table)), -1)
dist_matrix = np.zeros((len(node_idx_table), len(node_idx_table)))
for station_1 in station_list:
    for station_2 in station_list:
        if station_1 == station_2:
            continue
        dist = dist_on_sphere(coordinates[station_1], coordinates[station_2])
        if dist < 0.095:
            label_type = 1
        else:
            label_type = 0
        #elif dist < 0.145:
        #    label_type = 1
        #elif dist < 0.2:
        #    label_type = 2
        #else:
        #    label_type = 3
        dist_matrix[node_idx_table[station_1]][node_idx_table[station_2]] = label_type
print(dist_matrix)

import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

n, labels = connected_components(dist_matrix)
n, labels

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
  0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0

(1, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0], dtype=int32))

In [36]:
for ts in range(len(ts_idx_table)):
    adjacency = lil_matrix((len(node_idx_table),  len(node_idx_table)))
    for node_id_1 in range(len(node_idx_table)):
        for node_id_2 in range(len(node_idx_table)):
            if dist_matrix[node_id_1][node_id_2] == 1:
                adjacency[node_id_1, node_id_2] = 1
    mmwrite(OutputDir + '/adjacency' + str(ts), adjacency)

In [ ]:
dist_matrix.sum()

In [ ]:
df_pm25 = pd.read_csv(InputDir + "/PM2.5.csv")
df_pm25.head()

In [ ]:
df_pm25.isnull().sum()

In [ ]:
# 2015以降で全てのステーションが欠損値であるインデックスを保持、前処理時にサンプルに含めないようにする。
#df_missing = df_pm25.drop("时间", axis=1)[6288:].isnull().all(axis=1)
#len(list(df_missing[df_missing == True].index))

In [ ]:
# 列ごとに線形補完、2015年以降を使う（〜3/24）
df_pm25 = df_pm25.interpolate()[6288:]
df_pm25.head()

In [ ]:
df_char = pd.read_csv(InputDir + "/Characteristics.csv")
df_char

In [ ]:
station_list = ['HK', 'JA', 'PT', 'LW', 'XH', 'YP', 'CS', 'PD', 'ZJ', 'QP']
station_id = {}
for i in range(len(df_char)):
    station_id[df_char['Abbr'][i]] = df_char['ID'][i]
print(station_list)
print(station_id)

In [ ]:
df_feature = pd.read_csv(InputDir + "/feature/2015.csv")

In [ ]:
# 上海のステーションと2015年3月24日までの特徴量を抽出
df_station_feature = {}
conc = []
for station, ID in station_id.items():
    df = df_feature.query('time < "2015-03-25 00:00:00" and name =="' + ID + '"')
    df_station_feature[station] = df
    conc.append(df)

df_feature = pd.concat(conc)
df_feature

In [ ]:
node_idx_table = {}
for idx, node in enumerate(station_list):
    node_idx_table[node] = idx
node_idx_table

In [ ]:
rows = []
for node, idx in node_idx_table.items():
    rows.append([node, idx])
df = pd.DataFrame(rows, columns=['node', 'ID'])
df.to_csv(OutputDir + '/node_idx.csv', index=False, encoding='utf_8_sig')

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 12, 12
df_pm25.hist()

In [ ]:
rcParams['figure.figsize'] = 10, 6
sns.boxplot(data=df_pm25, whis=1.5)

In [ ]:
df_pm25.set_index('时间').plot()

In [ ]:
rcParams['figure.figsize'] = 12, 12
df_feature[['wind_u', 'wind_v', 'temperature', 'humidity', 'wind_speed',
       'wind_direction', 'pressure']].hist()

In [ ]:
rcParams['figure.figsize'] = 10, 6
sns.boxplot(data=df_feature[['wind_u', 'wind_v', 'temperature', 'humidity', 'wind_speed',
       'wind_direction']], whis=1.5)

In [ ]:
sns.boxplot(data=df_feature[['pressure']], whis=1.5)

In [ ]:
df_feature = df_feature[['wind_u', 'wind_v', 'temperature', 'humidity', 'wind_speed','pressure']]

wind_u_mean = df_feature['wind_u'].values.mean()
wind_u_std = df_feature['wind_u'].values.std()
wind_u_min = df_feature['wind_u'].values.min()
wind_u_max = df_feature['wind_u'].values.max()

wind_v_mean = df_feature['wind_v'].values.mean()
wind_v_std = df_feature['wind_v'].values.std()
wind_v_min = df_feature['wind_v'].values.min()
wind_v_max = df_feature['wind_v'].values.max()

temperature_mean = df_feature['temperature'].values.mean()
temperature_std = df_feature['temperature'].values.std()
temperature_min = df_feature['temperature'].values.min()
temperature_max = df_feature['temperature'].values.max()

humidity_mean = df_feature['humidity'].values.mean()
humidity_std = df_feature['humidity'].values.std()
humidity_min = df_feature['humidity'].values.min()
humidity_max = df_feature['humidity'].values.max()

wind_speed_mean = df_feature['wind_speed'].values.mean()
wind_speed_std = df_feature['wind_speed'].values.std()
wind_speed_min = df_feature['wind_speed'].values.min()
wind_speed_max = df_feature['wind_speed'].values.max()

pressure_mean = df_feature['pressure'].values.mean()
pressure_std = df_feature['pressure'].values.std()
pressure_min = df_feature['pressure'].values.min()
pressure_max = df_feature['pressure'].values.max()

In [ ]:
df_pm25_drop = df_pm25.drop("时间", axis=1)
pm25_mean = df_pm25_drop.values.mean()
pm25_std = df_pm25_drop.values.std()
pm25_min = df_pm25_drop.values.min()
pm25_max = df_pm25_drop.values.max()
print(pm25_mean)
print(pm25_std)
print(pm25_min)
print(pm25_max)
df_pm25_drop.head()

In [ ]:
columns = ['name', 'ID', 'range0', 'range1', 'type', 'primary', 'mean', 'std', 'min', 'max', 'class0', 'class1']

rows = []
row = ['sensor type', 0, 0, 1, 'VECTOR', True, False, False, False, False, 'fix', 'mobile']
rows.append(row)
row = ['PM2.5', 1, 2, 2, 'SCALAR', False, pm25_mean, pm25_std, pm25_min, pm25_max, False, False]
rows.append(row)
row = ['wind_u', 2, 3, 3, 'SCALAR', False, wind_u_mean, wind_u_std, wind_u_min, wind_u_max, False, False]
rows.append(row)
row = ['wind_v', 3, 4, 4, 'SCALAR', False, wind_v_mean, wind_v_std, wind_v_min, wind_v_max, False, False]
rows.append(row)
row = ['temperature', 4, 5, 5, 'SCALAR', False, temperature_mean, temperature_std, temperature_min, temperature_max, False, False]
rows.append(row)
row = ['humidity', 5, 6, 6, 'SCALAR', False, humidity_mean, humidity_std, humidity_min, humidity_max, False, False]
rows.append(row)
row = ['wind_speed', 6, 7, 7, 'SCALAR', False, wind_speed_mean, wind_speed_std, wind_speed_min, wind_speed_max, False, False]
rows.append(row)
row = ['pressure', 7, 8, 8, 'SCALAR', False, pressure_mean, pressure_std, pressure_min, pressure_max, False, False]
rows.append(row)

df = pd.DataFrame(rows, columns=columns)
df.to_csv(OutputDir + '/attribute_idx.csv', index=False, encoding='utf_8_sig')

In [ ]:
dates = []
for date in df_pm25['时间']:
    dates.append(date)

In [ ]:
rows = []
for idx, date in enumerate(dates):
    rows.append([date, idx])
df = pd.DataFrame(rows, columns=['TS', 'ID'])
df.to_csv(OutputDir + '/ts_idx.csv', index=False, encoding='utf_8_sig')

In [ ]:
# pandas 高速化 https://kunai-lab.hatenablog.jp/entry/2018/04/08/134924
df = pd.read_csv(OutputDir + '/ts_idx.csv')
df_TS = df.TS.values
df_ID = df.ID.values
ts_idx_table = {}
for row in range(df.shape[0]):
    ts_idx_table[df_TS[row]] = df_ID[row]

In [ ]:
exist_table = np.ones((len(node_idx_table), len(ts_idx_table)))
np.save(OutputDir + '/exist_table', exist_table)

In [ ]:
for ts in range(len(ts_idx_table)):
    node_attribute = np.zeros((len(node_idx_table), 9))
    for node_id in range(len(node_idx_table)):
        node_attribute[node_id][0] = 1 # primary class、固定ステーションを表す。
        node_attribute[node_id][2] = (df_pm25_drop.values[ts][node_id] - pm25_mean) / pm25_std
        node_attribute[node_id][3] = (df_feature['wind_u'].values[ts] - wind_u_mean) / wind_u_std
        node_attribute[node_id][4] = (df_feature['wind_v'].values[ts] - wind_v_mean) / wind_v_std
        node_attribute[node_id][5] = (df_feature['temperature'].values[ts] - temperature_mean) / temperature_std
        node_attribute[node_id][6] = (df_feature['humidity'].values[ts] - humidity_mean) / humidity_std
        node_attribute[node_id][7] = (df_feature['wind_speed'].values[ts] - wind_speed_mean) / wind_speed_std
        node_attribute[node_id][8] = (df_feature['pressure'].values[ts] - pressure_mean) / pressure_std
    np.save(OutputDir + '/node_attribute' + str(ts), node_attribute)

In [ ]:
# 緯度・経度から2点間の距離を求める
#https://qiita.com/s-wakaba/items/e12f2a575b6885579df7
from math import sin, cos, acos, radians
earth_rad = 6378.137

def latlng_to_xyz(lat, lng):
    rlat, rlng = radians(lat), radians(lng)
    coslat = cos(rlat)
    return coslat*cos(rlng), coslat*sin(rlng), sin(rlat)

def dist_on_sphere(pos0, pos1, radius=earth_rad):
    xyz0, xyz1 = latlng_to_xyz(*pos0), latlng_to_xyz(*pos1)
    return acos(sum(x * y for x, y in zip(xyz0, xyz1)))*radius

In [ ]:
coordinates_list = [ (df_char["Coordinates: E"][node_id], df_char["Coordinates: N"][node_id]) for node_id in range(len(node_idx_table))]
coordinates = {station_list[node_id]: coordinates_list[node_id] for node_id in range(len(node_idx_table))}
coordinates

In [ ]:
dist_list =[]
for station_1 in station_list:
    for station_2 in station_list[station_list.index(station_1):]:
        if station_1 == station_2:
            continue
        dist = dist_on_sphere(coordinates[station_1], coordinates[station_2])
        dist_list.append(dist)
dist_list = np.array(dist_list)
print("距離パーセンタイル")
print("000%", np.percentile(dist_list, 0))
print("025%", np.percentile(dist_list, 25))
print("050%", np.percentile(dist_list, 50))
print("075%", np.percentile(dist_list, 75))
print("100%", np.percentile(dist_list, 100))

In [ ]:
flag25, flag50, flag75, flag100 = 0, 0, 0, 0
adj_max_sum = len(node_idx_table) * len(node_idx_table) -  len(node_idx_table)
print("隣接行列dense率と距離閾値")
for thres in range(5800):
    thres = thres / 100.0
    adj = np.zeros((len(node_idx_table), len(node_idx_table)))
    for station_1 in station_list:
        for station_2 in station_list:
            if station_1 == station_2:
                continue
            dist = dist_on_sphere(coordinates[station_1], coordinates[station_2])
            if dist < thres:
                adj[node_idx_table[station_1]][node_idx_table[station_2]] = 1
    dense_rate = adj.sum() / adj_max_sum
    if flag25==0 and dense_rate > 0.25:
        flag25 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)
    if flag50==0 and dense_rate > 0.5:
        flag50 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)
    if flag75==0 and dense_rate > 0.75:
        flag75 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)
    if flag100==0 and dense_rate >= 1:
        flag100 = 1
        print(str(int(dense_rate*100))+"%,  <", thres)

In [ ]:
dist_matrix = np.full((len(node_idx_table), len(node_idx_table)), -1)
for station_1 in station_list:
    for station_2 in station_list:
        if station_1 == station_2:
            continue
        dist = dist_on_sphere(coordinates[station_1], coordinates[station_2])
        if dist < 8.84:
            label_type = 0
        elif dist < 13.49:
            label_type = 1
        elif dist < 26.76:
            label_type = 2
        else:
            label_type = 3
        dist_matrix[node_idx_table[station_1]][node_idx_table[station_2]] = label_type
print(dist_matrix)

In [ ]:
for ts in range(len(ts_idx_table)):
    adjacency = lil_matrix((len(node_idx_table), 4 * len(node_idx_table)))
    for node_id_1 in range(len(node_idx_table)):
        for node_id_2 in range(len(node_idx_table)):
            if node_id_1 == node_id_2:
                continue
            label_edge_type = dist_matrix[node_id_1][node_id_2]
            adjacency[node_id_1, label_edge_type * len(node_idx_table) + node_id_2] = 1
    mmwrite(OutputDir + '/adjacency' + str(ts), adjacency)

In [ ]:
a = np.zeros((1000,27,8,1))
a.shape

In [ ]:
a.sum(axis=1).shape

In [ ]:
a.sum(axis=0).shape

In [ ]:
a = np.array([[[0,1,2],[3,4,5],[6,7,8]],[[0,1,2],[3,4,5],[6,7,8]]])
a

In [ ]:
a.mean(axis=0)

In [ ]:
a.sum(axis=1)

In [1]:
86014 * 15

1290210

In [2]:
1290210 / (60 * 24)

895.9791666666666

In [26]:
def n_sample(days):
    return days * (4*24)

def start_idx(days):
    all_size = 109514
    end = -15000
    for now_start in range(all_size + end):
        train_size = len(list(range(all_size))[now_start:end])
        if train_size == n_sample(days):
            return now_start

In [27]:
print("2年ちょい、今のやつ", start_idx(896))

2年ちょい、今のやつ 8498


In [28]:
print("半年", start_idx(int(365/2) * 1))

半年 77042


In [29]:
print("1年", start_idx(int(365/2) * 2))

1年 59570


In [30]:
print("1年半", start_idx(int(365/2) * 3))

1年半 42098


In [31]:
print("2年", start_idx(int(365/2) * 4))

2年 24626


In [32]:
print("2年半", start_idx(int(365/2) * 5))

2年半 7154


In [42]:
print("3年", start_idx(int(365/2) * 6))

3年 None


In [47]:
print("もともと", (109514-15000)*15 / (60 * 24), "日分しかtrainを取れない")
print("3年だと", int(365/2)*6, "分必要")

もともと 984.5208333333334 日分しかtrainを取れない
3年だと 1092 分必要


In [59]:
now_train = list(range(109514))[8500:-15000]
print("半年", len(now_train[77042-8500:]))
print("1年", len(now_train[59570-8500:]))
print("1年半", len(now_train[42098-8500:]))
print("2年", len(now_train[24626-8500:]))
print("いま", len(now_train[8500-8500:]))

半年 17472
1年 34944
1年半 52416
2年 69888
いま 86014
